In [2]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pylab
import numpy as np
import scipy.io
import scipy.stats
import datetime
import pandas as pd
import matplotlib as mpl
import math

%matplotlib inline
from scipy.stats import stats
from IPython.display import display
from glob import glob

In [3]:
# load date format and using 2011-01-24 as the day 1
from datetime import datetime
date_format = "%Y-%m-%d"
day1 = datetime.strptime('2011-01-23',date_format)

## Load original csv file as dataframe

In [4]:
emp_retention = pd.read_csv('./employee_retention_data.csv',header = 0) # emp is short for employee 

## Making a table with all companies and head count for all days

In [5]:
# Sort all data with company id and join date
emp_retention_sorted = emp_retention.sort_values(['company_id','join_date'])

# Create head count with +1 and -1 
emp_retention_sorted['join_date_head_count'] = 1
emp_retention_sorted['quit_date_head_count'] = -1

all_company_headcount = pd.DataFrame()

for company_num in range(1,13): # Total 12 companies 
    each_company = emp_retention_sorted[emp_retention_sorted.company_id == company_num]
    
    topdf = each_company[['join_date','join_date_head_count']]
    bottomdf = each_company[['quit_date','quit_date_head_count']]
    topdf.columns = ['date' , 'head_count']
    bottomdf.columns = topdf.columns
    # append join date with quit date columns
    day_headcount = topdf.append([bottomdf])
    day_headcount.sort_values('date',inplace = True)
    # caculate the head count for each date 
    agg_day_headcount = day_headcount.groupby('date').sum()
    
    day_column = []

    for each_date in agg_day_headcount.index:
        # Convert year-month-date format to day number 
        delta = datetime.strptime(each_date,date_format) - day1
        day_column.append(delta.days)

    agg_day_headcount['day_num'] = day_column
    agg_day_headcount['employee_headcount'] = agg_day_headcount['head_count'].cumsum()
    agg_day_headcount.drop('head_count', 1, inplace = True)
    agg_day_headcount['company_id'] = company_num
    all_company_headcount = all_company_headcount.append([agg_day_headcount], ignore_index = True)

all_company_headcount.sort_values('day_num',inplace = True)
all_company_headcount.set_index('company_id',inplace = True)

In [62]:
# Saving the csv file 
all_company_headcount.to_csv('./all_company_headcount.csv', header = True, index = True)

## Clean up the features for emp_retention_sorted

In [6]:
# Reload the data again to avoid unwanted changes 
emp_retention = pd.read_csv('./employee_retention_data.csv',header = 0) # emp is short for employee 
emp_retention_sorted = emp_retention.sort_values(['company_id','join_date'])
emp_retention_sorted.drop('employee_id', 1, inplace = True)
emp_retention_sorted.set_index('company_id', inplace = True)

### Convert date format to day # starting 2011/1/23

In [7]:
join_date_column = []
for each_date in emp_retention_sorted['join_date'].values:
    delta1 = datetime.strptime(each_date,date_format) - day1
    join_date_column.append(delta1.days)

In [13]:
quit_date_column = []
for each_date in emp_retention_sorted['quit_date'].values:
    # Check whether quit date is NaN, if so put quit day to be 20000 days (55 years)
    if each_date != each_date:
        quit_date_column.append(20000)  
    else:
        delta2 = datetime.strptime(each_date,date_format) - day1
        quit_date_column.append(delta2.days)

In [15]:
emp_retention_sorted['join_day'] = join_date_column
emp_retention_sorted['quit_day'] = quit_date_column

In [19]:
emp_retention_sorted

,dept,seniority,salary,join_date,quit_date,join_day,quit_day
company_id,,,,,,,
1,customer_service,19,108000.0,2011-01-24,2012-02-24,1,397
1,customer_service,20,121000.0,2011-01-24,2012-01-06,1,348
1,data_science,15,244000.0,2011-01-24,2014-05-09,1,1202
1,sales,28,191000.0,2011-01-24,2013-03-08,1,775
1,customer_service,9,77000.0,2011-01-24,2012-11-09,1,656
1,sales,14,137000.0,2011-01-24,2014-01-03,1,1076
1,data_science,1,74000.0,2011-01-24,2012-08-31,1,586
1,data_science,17,342000.0,2011-01-24,2015-01-16,1,1454
1,sales,17,218000.0,2011-01-24,2014-01-24,1,1097
